# Hate Speech Detector - EN - Features extraction for Conv. & LSTM model

Based on [this notebook](https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/classifier/final_classifier.ipynb).

In [1]:
import os
import pandas as pd
import csv
import numpy as np
import pickle
from klepto.archives import dir_archive
import sys
import nltk
import string
import re
import fasttext
from pymagnitude import *

from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from tqdm.notebook import tqdm
%matplotlib inline

In [2]:
MODEL = 'lstm'

## Davidson et al. data loading

### Classes pre:
    0 - hate speech
    1 - offensive language
    2 - neither

In [3]:
if not os.path.exists('hsd/Reddit/perfect_data.pkl'):
    tweets, labels = [], []
    with open('hsd/Reddit/clean_data.csv', 'r') as f:
        for d in list(csv.reader(f))[1:]:
            tweets.append(d[1])  # tweet
            labels.append(d[2])  # class
    with open('hsd/Reddit/perfect_data.pkl', 'wb') as f:
        def chcl(c):
            return 0 if c=='2' else 1
        labels = list(map(chcl, labels))
        pickle.dump((tweets, labels), f)
else:
    with open('hsd/Reddit/perfect_data.pkl', 'rb') as f:
        tweets, labels = pickle.load(f)

### Classes post:
    0 - no hate
    1 - hate speech

In [4]:
print('Tweets: {}'.format(len(tweets)))
print('Labels: {}'.format(len(labels)))

Tweets: 55132
Labels: 55132


In [5]:
list(zip(tweets[:5], labels[:5]))

[("i joined gab to remind myself how retarded jew haters are. You wouldn't be typing on your abacus without them you retard.",
  1),
 ('This is what the left is really scared of.    https://redd.it/9rfkts', 0),
 ('That literally looks like a monkey. Why are we supposed to pretend it’s a person bc it’s wearing a red hat?',
  0),
 ('Dumb Cunt', 1),
 ('It makes you an asshole.', 0)]

## Features extraction

In [6]:
def pos_tagger(nltk_tag): 
    if nltk_tag.startswith('J'): 
        return wordnet.ADJ 
    elif nltk_tag.startswith('V'): 
        return wordnet.VERB 
    elif nltk_tag.startswith('N'): 
        return wordnet.NOUN 
    elif nltk_tag.startswith('R'): 
        return wordnet.ADV 
    else:           
        return None

def word_tokenization(tweet):
    lemmatizer = WordNetLemmatizer() 
    tokens = word_tokenize(tweet)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    tags = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(w[0]) if pos_tagger(w[1]) is None else lemmatizer.lemmatize(w[0], pos_tagger(w[1])) for w in tags]
    tags = [x[1] for x in tags]
    return words, tags

def preprocess(text_string):
    """
    Accepts a text string and replaces:
    1) urls with URLHERE
    2) lots of whitespace with one instance
    3) mentions with MENTIONHERE
    4) hashtags with HASHTAGHERE

    This allows us to get standardized counts of urls and mentions
    Without caring about specific people mentioned
    """
    space_pattern = '\s+'
    giant_url_regex = ('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|'
        '[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    mention_regex = '@[\w\-]+'
    hashtag_regex = '#[\w\-]+'
    parsed_text = text_string.encode('ascii', 'ignore').decode('ascii')
    parsed_text = re.sub(space_pattern, ' ', parsed_text)
    parsed_text = re.sub(giant_url_regex, '', parsed_text)
    parsed_text = re.sub(mention_regex, '', parsed_text)
    parsed_text = parsed_text.strip('#')
    list_words, tag_list = word_tokenization(parsed_text)
    parsed_text = " ".join(list_words)
    tag_str = ' '.join(tag_list)
    return parsed_text, tag_str

def basic_tokenize(tweet):
    tweet = " ".join(re.split(" ", tweet.lower())).strip()
    return tweet.split()

# def get_pos_string(tweet):
#     text = preprocess(tweet)
#     tokens = word_tokenize(text)
#     tags = nltk.pos_tag(tokens)
#     tag_list = [x[1] for x in tags]
#     tag_str = ' '.join(tag_list)
    
    # return tag_str

def pad_words(words, length):
    if len(words) >= length:
        return words[:length]
    else:
        additional = length - len(words)
        return words + ['EMPTY']*additional

In [7]:
# from keras.preprocessing.text import Tokenizer
# from keras.preprocessing import sequence
# max_features = 500000
# posts = [preprocess(tweet)[0] for tweet in tweets]
# tokenizer = Tokenizer(num_words = max_features)
# tokenizer.fit_on_texts(list(posts))
# tokenized_train = tokenizer.texts_to_sequences(posts)
# word_index = tokenizer.word_index
# print("dictionary size: ", len(word_index))
# # pad_tweets = sequence.pad_sequences(tokenized_train, maxlen=19)

### Median sentences length

In [8]:
def median_sentences_length(data):
    all_lengths, wt_lengths, pos_lengths = [], [], []
    for d in data:
        sentence, pos_string = preprocess(d)
        # sentence = preprocess(d)
        # pos_string = get_pos_string(d)
        all_lengths.append(len(sentence.split(' ')))
        all_lengths.append(len(pos_string.split(' ')))
        wt_lengths.append(len(sentence.split(' ')))
        pos_lengths.append(len(pos_string.split(' ')))
    
    return int(np.median(all_lengths)), int(np.mean(wt_lengths) + np.std(wt_lengths)), int(np.median(pos_lengths)), int(np.max(wt_lengths))

In [9]:
opt_length, opt_wt_length, opt_pos_length, max_length = median_sentences_length(tweets)
dim = 6*20 if MODEL == 'conv' else 300

print('Optimal all length: {}'.format(opt_length))
print('Optimal sentence length: {}'.format(opt_wt_length))
print('Optimal pos sentence length: {}'.format(opt_pos_length))
print('Maximum Length: {}'.format(max_length))

Optimal all length: 12
Optimal sentence length: 48
Optimal pos sentence length: 12
Maximum Length: 787


### Supervised fastText wordtokens training

In [10]:
if not os.path.exists('hsd/Reddit/fasttext.ft'):
    with open('hsd/Reddit/fasttext.ft', 'a') as f:
        for t, l in list(zip(tweets, labels)):
            text = preprocess(t)[0]
            if len(text) > 0:
                f.write('__label__{} {}\n'.format(l, text))

# load fasttext model or train & save if none
if os.path.exists('hsd/Reddit/fasttext_{}.bin'.format(MODEL)):
    ft_model = fasttext.load_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))
else:
    ft_model = fasttext.train_supervised('hsd/Reddit/fasttext.ft',
                                         lr=0.5, epoch=50, wordNgrams=3, dim=dim)
    ft_model.save_model('hsd/Reddit/fasttext_{}.bin'.format(MODEL))

### Wordtoken features

In [11]:
def get_wordtoken_fts(data, length):
    
    sentences_words = []
    for d in data:
        sentence = preprocess(d)[0]
        sentences_words.append(sentence.split(' '))
    
    sentences_words = [pad_words(sw, length) for sw in sentences_words]
    vecs = Magnitude(MagnitudeUtils.download_model('http://magnitude.plasticity.ai/fasttext/medium/crawl-300d-2M.magnitude'))
    
    ft_matrices = []
    for sw in sentences_words:
        ft_matrix = []
        for w in sw:
            ft_matrix.append(vecs.query(w))
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [12]:
wordtoken_features = get_wordtoken_fts(tweets, opt_wt_length)
ft_model = None

In [13]:
wordtoken_features[0]

02, -1.353419e-01, -1.226980e-02, -4.472660e-02,
        -2.709990e-02, -1.305760e-02, -1.012307e-01, -4.061040e-02,
        -3.842430e-02, -5.398310e-02, -9.037890e-02, -8.273740e-02,
        -1.813880e-02,  6.347600e-02,  4.027560e-02, -2.903000e-02,
         6.583930e-02, -2.150660e-02,  1.118660e-02, -1.063510e-02,
        -2.668630e-02, -2.040369e-01, -2.841940e-02,  1.101327e-01,
         9.693720e-02, -6.322000e-03, -4.594770e-02,  6.676500e-03,
         1.398300e-03,  5.055630e-02, -1.398300e-03,  1.970000e-05,
        -3.933030e-02,  1.697680e-02,  1.268340e-02, -8.669600e-02,
        -8.850790e-02,  7.340210e-02,  3.470200e-02, -2.838000e-02,
         1.565730e-02, -7.631690e-02, -7.880000e-05, -1.867060e-02,
        -5.837500e-02, -5.467240e-02,  7.570640e-02,  8.714900e-02,
        -1.476117e-01, -8.525830e-02,  4.135900e-03,  7.255520e-02,
         4.005900e-02, -3.629730e-02,  1.191530e-02,  5.252570e-02,
         8.795650e-02,  5.457400e-02, -1.208072e-01,  3.354000e-02,

### Supervised fastText pos training

In [14]:
# if not os.path.exists('hsd/Reddit/fasttext_pos.ft'):
#     with open('hsd/Reddit/fasttext_pos.ft', 'a') as f:
#         for t, l in list(zip(tweets, labels)):
#             f.write('__label__{} {}\n'.format(l, preprocess(t)[1]))

# # load fasttext pos model or train & save if none
# if os.path.exists('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL)):
#     ft_pos_model = fasttext.load_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))
# else:
#     ft_pos_model = fasttext.train_supervised('hsd/Reddit/fasttext_pos.ft',
#                                              lr=0.5, epoch=50, wordNgrams=3, dim=dim)
#     ft_pos_model.save_model('hsd/Reddit/fasttext_pos_{}.bin'.format(MODEL))

### Part of speech (PoS) features

In [15]:
def get_pos_fts(data, length):

    #Get POS tags for tweets and save as a string
    pos_sentences = []
    for d in data:
        pos_string = preprocess(d)[1]
        pos_sentences.append(pos_string)
        
        
    pos_tags = []
    for ps in pos_sentences:
        pos_tags.append(ps.split(' '))
    
    pos_tags = [pad_words(pt, length) for pt in pos_tags]
    
    ft_matrices = []
    for pt in pos_tags:
        ft_matrix = []
        for t in pt:
            ft_matrix.append(ft_pos_model[t])
        ft_matrices.append(ft_matrix)
    
    return ft_matrices

In [16]:
# pos_features = get_pos_fts(tweets, opt_pos_length)
tweets = None
# ft_pos_model = None

In [17]:
# pos_features[0]

In [18]:
np.array(wordtoken_features)

array([[[-0.0147371 , -0.0524712 , -0.0297504 , ..., -0.0042178 ,
          0.0668819 , -0.0136827 ],
        [-0.0265963 , -0.0167253 , -0.0688487 , ..., -0.0375365 ,
         -0.0471593 , -0.0073507 ],
        [ 0.0004202 , -0.0573342 ,  0.0080893 , ..., -0.0176494 ,
         -0.0299409 , -0.0282863 ],
        ...,
        [ 0.0013195 , -0.0405317 , -0.0390152 , ..., -0.0215854 ,
         -0.0513047 ,  0.0802165 ],
        [ 0.0013195 , -0.0405317 , -0.0390152 , ..., -0.0215854 ,
         -0.0513047 ,  0.0802165 ],
        [ 0.0013195 , -0.0405317 , -0.0390152 , ..., -0.0215854 ,
         -0.0513047 ,  0.0802165 ]],

       [[-0.0842272 ,  0.0433275 ,  0.0215392 , ...,  0.0525097 ,
          0.0721814 ,  0.0056338 ],
        [ 0.0129542 , -0.18601809, -0.0117232 , ..., -0.0946064 ,
          0.025498  ,  0.0967165 ],
        [-0.0357206 , -0.0546785 ,  0.0011175 , ..., -0.0987406 ,
          0.0186785 , -0.0030732 ],
        ...,
        [ 0.0013195 , -0.0405317 , -0.0390152 , ..., -

In [19]:
words_token_shape = np.array(wordtoken_features).shape[1]

In [ ]:
# np.array(pos_features).shape

### All features

In [ ]:
# #Now join them all up
features = np.array(wordtoken_features)
#features = np.concatenate([wordtoken_features, pos_features], axis=1)
wordtoken_features = None
pos_features = None
# ft_pos_model = None

In [ ]:
features.shape

In [ ]:
features[0]


## Save features & labels

In [ ]:
archive = dir_archive('hsd/Reddit/X_y_{}'.format(MODEL), {'features': features, 'labels': labels,
                                                                'wt_num': words_token_shape}, serialized=True)
archive.dump()
del archive